In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
r = requests.get('https://www.basketball-reference.com/leagues/NBA_2021_advanced.html')
soup = bs(r.content)
soup.findAll('tr', limit=2)
headers = [th.getText() 
           for th in soup.findAll('tr', limit=2)[0].findAll('th')]

headers = headers[1:]

rows = soup.findAll('tr')[1:]
players = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]
stats = pd.DataFrame(players, columns = headers)
stats=stats.drop_duplicates(subset=['Player'])
stats

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP
0,Precious Achiuwa,PF,21,MIA,12,208,15.9,.615,.000,.603,...,,0.3,0.2,0.5,.114,,-2.2,-1.8,-4.0,-0.1
1,Jaylen Adams,PG,24,MIL,1,1,-39.2,.000,1.000,.000,...,,0.0,0.0,0.0,-0.987,,-36.5,-20.9,-57.4,0.0
2,Steven Adams,C,27,NOP,13,398,17.8,.627,.000,.435,...,,0.7,0.4,1.2,.144,,0.8,0.4,1.2,0.3
3,Bam Adebayo,C,23,MIA,10,313,24.3,.714,.009,.626,...,,1.0,0.4,1.4,.220,,2.5,3.0,5.5,0.6
4,LaMarcus Aldridge,C,35,SAS,11,307,13.2,.495,.266,.084,...,,-0.1,0.3,0.2,.034,,-2.1,-1.1,-3.2,-0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,Delon Wright,SG,28,DET,13,336,12.6,.489,.318,.271,...,,0.3,0.2,0.5,.077,,-0.6,0.2,-0.4,0.1
492,Thaddeus Young,PF,32,CHI,10,235,14.8,.607,.175,.163,...,,0.2,0.1,0.4,.075,,-1.2,-0.9,-2.1,0.0
493,Trae Young,PG,22,ATL,13,425,19.8,.551,.340,.586,...,,0.8,0.4,1.2,.134,,3.8,-2.0,1.8,0.4
494,Cody Zeller,C,28,CHO,1,19,14.4,.466,.333,.167,...,,0.0,0.1,0.0,.049,,-3.9,1.6,-2.3,0.0


In [2]:
r2 = requests.get('https://www.basketball-reference.com/contracts/players.html')
soup2 = bs(r2.content)
soup2.findAll('tr', limit=2)
headers2 = [th.getText() 
           for th in soup2.findAll('tr', limit=2)[1].findAll('th')]

headers2 = headers2[1:]
rows = soup2.findAll('tr')[1:]
playersContracts = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]
contracts = pd.DataFrame(playersContracts, columns = headers2)
contracts.head()

,Player,Tm,2020-21,2021-22,2022-23,2023-24,2024-25,2025-26,Signed Using,Guaranteed
0,None,None,None,None,None,None,None,None,None,None
1,Stephen Curry,GSW,"$43,006,362","$45,780,966",,,,,Bird Rights,"$88,787,328"
2,Russell Westbrook,WAS,"$41,358,814","$44,211,146","$47,063,478",,,,Bird Rights,"$85,569,960"
3,Chris Paul,PHO,"$41,358,814","$44,211,146",,,,,Bird Rights,"$41,358,814"
4,John Wall,HOU,"$41,254,920","$44,310,840","$47,366,760",,,,Bird Rights,"$85,565,760"


In [3]:
contracts=contracts.drop(contracts.index[0])
contracts=contracts.dropna()
contracts=contracts.sort_values('Player', ascending = True)
contracts=contracts.drop(['Tm', '2021-22','2022-23','2023-24','2024-25', 'Signed Using','Guaranteed', '2025-26'],axis=1)

#We need to change the value of contract to integer and drop '$',',','.' characters
contracts['2020-21']=contracts['2020-21'].str.replace(r'\$',"")
contracts['2020-21']=contracts['2020-21'].str.replace(',',"")
contracts['2020-21']=contracts['2020-21'].str.replace('.',"")
contracts=contracts.drop_duplicates(subset=['Player'])
contracts

,Player,2020-21
516,A.J. Hammons,350087
67,Aaron Gordon,18136364
342,Aaron Holiday,2345640
289,Aaron Nesmith,3458400
412,Abdel Nader,1752950
...,...,...
482,Zach Norvell,1445697
341,Zeke Nnaji,2379840
294,Zhaire Smith,3204600
128,Zion Williamson,10245480


In [11]:
nba=stats.merge(contracts, how='inner', on='Player')
nba

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP,2020-21
0,Precious Achiuwa,PF,21,MIA,12,208,15.9,.615,.000,.603,...,0.3,0.2,0.5,.114,,-2.2,-1.8,-4.0,-0.1,2582160
1,Steven Adams,C,27,NOP,13,398,17.8,.627,.000,.435,...,0.7,0.4,1.2,.144,,0.8,0.4,1.2,0.3,27528090
2,Bam Adebayo,C,23,MIA,10,313,24.3,.714,.009,.626,...,1.0,0.4,1.4,.220,,2.5,3.0,5.5,0.6,5115492
3,LaMarcus Aldridge,C,35,SAS,11,307,13.2,.495,.266,.084,...,-0.1,0.3,0.2,.034,,-2.1,-1.1,-3.2,-0.1,24000000
4,Nickeil Alexander-Walker,SG,22,NOP,11,221,14.5,.546,.425,.123,...,0.0,0.2,0.3,.059,,-1.2,0.2,-1.0,0.1,3113160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,Delon Wright,SG,28,DET,13,336,12.6,.489,.318,.271,...,0.3,0.2,0.5,.077,,-0.6,0.2,-0.4,0.1,9000000
409,Thaddeus Young,PF,32,CHI,10,235,14.8,.607,.175,.163,...,0.2,0.1,0.4,.075,,-1.2,-0.9,-2.1,0.0,13545000
410,Trae Young,PG,22,ATL,13,425,19.8,.551,.340,.586,...,0.8,0.4,1.2,.134,,3.8,-2.0,1.8,0.4,6571800
411,Cody Zeller,C,28,CHO,1,19,14.4,.466,.333,.167,...,0.0,0.1,0.0,.049,,-3.9,1.6,-2.3,0.0,15415730


In [9]:
import csv
nba.to_csv('/Users/Piotrek/Desktop/nbaData.csv')